#   Sentimental Analysis using NLP

### 01.Finding the Problem
Identifying the Sentiments from the text

### 02.Collecting Dataset

### 03.Load & Summarize dataset

In [1]:
import pandas as pd
dataset = pd.read_csv('dataset.csv')
dataset.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


### 04.Segregating Dataset into X & Y

In [2]:
# X=dataset['text']
# y=dataset['airline_sentiment']
features=dataset.iloc[:,10].values ## input-X
labels=dataset.iloc[:,1].values ## output-y

print(features.shape)
print(labels.shape)
print(labels)


(14640,)
(14640,)
['neutral' 'positive' 'neutral' ... 'neutral' 'negative' 'neutral']


### 05.Removing the Special Characters

In [3]:
import re

processed_features = []

for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)



### 06.Feature Extraction from Text

In [4]:
import nltk
# nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

vectorizer = TfidfVectorizer(max_features=2500,min_df=7,max_df=0.8,stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(processed_features).toarray() ## text to vector numerical
print(processed_features.shape)
print(processed_features)


(14640, 2301)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### 07.Splitting the dataset into train and test data

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(processed_features,labels, test_size=0.2, random_state=0)

### 08.Model Training and load the algorithms

In [6]:
from sklearn.ensemble import RandomForestClassifier
text_classifier=RandomForestClassifier(n_estimators=200,random_state=0)
text_classifier.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200, random_state=0)

### 09.Model Prediction

In [7]:
y_pred = text_classifier.predict(X_test)
from sklearn.metrics import accuracy_score
print("Accuracy of the model on the test data: %.2f" % (accuracy_score(y_test, y_pred)*100))

Accuracy of the model on the test data: 75.99


In [9]:
y_pred = text_classifier.predict(X_test)
from sklearn import metrics
cm=metrics.confusion_matrix(y_test,y_pred)
cm

array([[1723,  108,   39],
       [ 326,  248,   40],
       [ 132,   58,  254]], dtype=int64)

In [10]:
#Sample notebook for Sentimental Analysis
new_input=["@virginamerica Well, I didn't #$ but NOW I DO!"]
new_input_vector=vectorizer.transform(new_input).toarray()
new_pred=text_classifier.predict(new_input_vector)
print(f"Predicted Sentiment: {new_pred[0]}")

Predicted Sentiment: positive


### svm

In [15]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_train,y_train)


MultinomialNB()

In [16]:
y_pred1 = model.predict(X_test)
from sklearn.metrics import accuracy_score
print("Accuracy of the model on the test data: %.2f" % (accuracy_score(y_test, y_pred1)*100))

Accuracy of the model on the test data: 75.82
